##### Students :
- Adrien Mainka 20130281
- Abdelmoneim AlKhalil

To be able to run this notebook, the data files needs to be unzipped and store in `./data/`

## Vocabulary Size

To measure the vocabulary size, we use a simple naive space tokenizer :<br> `tr ' ' '\n'` or `str.split()`

In [1]:
import time
from collections import Counter
from itertools import chain

def countInFile(filename):
    with open(filename) as f:
        return Counter(chain.from_iterable(map(str.split, f)))
counters = dict()

### train.en

In [2]:
# !cat data/train.en | tr ' ' '\n' | sort | uniq | wc -w
# 859973
start_time = time.time()
counters["train_en"] = countInFile("data/train.en")
print("vocab size = {:,}\t elapsed time = {:.2f} s".format(len(counters["train_en"]), time.time() - start_time))

vocab size = 860,818	 elapsed time = 24.22 s


### train.fr

In [16]:
# !cat data/train.fr | tr ' ' '\n' | sort | uniq | wc -w
# TOO SLOW

start_time = time.time()
counters["train_fr"] = countInFile("data/train.fr")
print("vocab size = {:,}\t elapsed time = {:.2f} s".format(len(counters["train_fr"]), time.time() - start_time))

vocab size = 3,310,267	 elapsed time = 338.46 s


### train.de

In [3]:
# !cat data/train.de | tr ' ' '\n' | sort | uniq | wc -w
start_time = time.time()
counters["train_de"] = countInFile("data/train.de")
print("vocab size = {:,}\t elapsed time = {:.2f} s".format(len(counters["train_de"]), time.time() - start_time))

vocab size = 1,713,522	 elapsed time = 31.78 s


## Number of words seen at least 100 times
### train.en

In [4]:
# !cat data/train.en | tr ' ' '\n' | sort | uniq -c | sort -k1 -nr | awk '$1 > 99' | wc -l
print("Number of words seenat least 100 times : {:,}".format(len([token for token in counters["train_en"] if counters["train_en"][token] >= 100])))

Number of words seenat least 100 times : 30,395


## train.fr

In [17]:
# !cat data/train.fr | tr ' ' '\n' | sort | uniq -c | sort -k1 -nr | awk '$1 > 99' | wc -l
print("Number of words seenat least 100 times : {:,}".format(len([token for token in counters["train_fr"] if counters["train_fr"][token] >= 100])))

Number of words seenat least 100 times : 124,291


### train.de

In [5]:
# !cat data/train.de | tr ' ' '\n' | sort | uniq -c | sort -k1 -nr | awk '$1 > 99' | wc -l
print("Number of words seenat least 100 times : {:,}".format(len([token for token in counters["train_de"] if counters["train_de"][token] >= 100])))

Number of words seenat least 100 times : 42,518


## Number of words seen in test but not in train
### train.en

In [14]:
counters["test_en"] = countInFile("data/test.en")
print("Number of words seen in test but not in train: {:,}".format(len([word for word in counters["test_en"] if not word in counters["train_en"]])))

Number of words seen in test but not in train: 427


### train.fr

In [18]:
counters["test_fr"] = countInFile("data/test.fr")
print("Number of words seen in test but not in train: {:,}".format(len([word for word in counters["test_fr"] if not word in counters["train_fr"]])))

Number of words seen in test but not in train: 132


### train.de

In [15]:
counters["test_de"] = countInFile("data/test.de")
print("Number of words seen in test but not in train: {:,}".format(len([word for word in counters["test_de"] if not word in counters["train_de"]])))

Number of words seen in test but not in train: 1,035
